# REDO - Homework 5 Assignment 2
yg833 - completed the incorrect notebook the first time around. This is a redo referencing the correct slides : https://serv.cusp.nyu.edu/~hvo/files/SQL_Lab.pdf

The skeleton notebook was provided by Dr. Bianco : https://github.com/fedhere/PUI2018_fb55/blob/master/HW5_fb55_session2/SQL_query.ipynb

## SQL CARTO QUERY FUNCTION
## Originally written for PUI2016_Python2
## Extra Credit: extra credit: make the function python 2 and 3 compatible so that it works on the PUI2016_Python3 kernel

##### This notebook is set up to link as a default to the fb55 account.  To turn in the homework use the same account you used in the lab ~~(hvt201)~~ with Mr. Kryaschev  and query the database that you were querying in class (citibike_feb_2015)

In [2]:
## queryCartoDB function based on the one shared via class slack but it didn't work for me so I had to adapt it

## pull in libs
import sys
import pandas as pd
import ast
import numpy as np

# define the sql source variable
sql_source = 'https://fb55.carto.com/api/v2/sql?q='

# find the version of python and import the necessary libraries as appropriate for the version
if sys.version_info[0] == 2:
    print("This notebook is running Python version", sys.version_info[0])
    import urllib2
    import urllib
    from StringIO import StringIO as io # for python 2 io is from StringIO
    
    def queryCartoDB(query, format='CSV', source=sql_source):
        '''
        queries carto datasets from a given carto account
        Arguements:
        query - string: a valid sql query string
        format - output format OPTIONAL (default CSV)
        source - a valid sql api endpoint (default carto fb55 account)
        Returns:
        the return of the sql query as a string
        Notes:
        designed for hte carto API, tested only with CSV return format
        '''
        data = urllib.urlencode({'format':format, 'q': query})
        try:
            response = urllib2.urlopen(source, data)
        except (urllib2.HTTPError, e):
            raise ValueError('\n'.join(ast.literal_eval(e.readline())['error']))
        except Exception:
            raise
        return response.read()

else:
    print("This notebook is running Python version", sys.version_info[0])
    
    import urllib
    from io import BytesIO as io # for python 3 io is from BytesIO
    
    def queryCartoDB(query, format='CSV', source=sql_source):
        '''
        queries carto datasets from a given carto account
        Arguements:
        query - string: a valid sql query string
        format - output format OPTIONAL (default CSV)
        source - a valid sql api endpoint (default carto fb55 account)
        Returns:
        the return of the sql query as a string
        Notes:
        designed for hte carto API, tested only with CSV return format
        '''
        data = urllib.parse.urlencode({'format': format, 'q': query}).encode("utf-8")
        try:
            response = urllib.request.urlopen(source, data)
            return response.read()
        except urllib.error.HTTPError as e:
            raise(ValueError('\n'.join(ast.literal_eval(e.readline())['error'])))

This notebook is running Python version 3


In [3]:
## define a function that gets the data from the cartodb
def get_data(query):
    try:
        return pd.read_csv(io(queryCartoDB(query)), sep = ',') # uses the function created in the kernel above
    except ValueError as v:
        print (str(v))

In [4]:
query = '''SELECT * FROM fb55.citibike; ''' # query string variable

In [5]:
# now get the data
table = get_data(query)
table.head(5)

,the_geom,start_station_name,the_geom_webmercator,field_1,start_station_longitude,tripduration,starttime,stoptime,start_station_id,cartodb_id,start_station_latitude,end_station_id,end_station_name,end_station_latitude,end_station_longitude,bikeid,usertype,birth_year,gender
0,NaN,E 20 St & Park Ave,NaN,175,-73.987520,1090,2015-02-01 01:23:00+00,2015-02-01 01:42:00+00,503,107,40.738274,229,Great Jones St,40.727434,-73.993790,19718,Subscriber,1961.0,1
1,NaN,W 43 St & 10 Ave,NaN,1159,-73.994618,682,2015-02-01 10:55:00+00,2015-02-01 11:07:00+00,515,1088,40.760094,490,8 Ave & W 33 St,40.751551,-73.993934,21501,Subscriber,1981.0,1
2,NaN,E 6 St & Avenue B,NaN,2827,-73.981854,751,2015-02-01 13:59:00+00,2015-02-01 14:11:00+00,317,2759,40.724537,466,W 25 St & 6 Ave,40.743954,-73.991449,14788,Subscriber,1990.0,1
3,NaN,E 12 St & 3 Ave,NaN,4961,-73.988900,272,2015-02-01 17:28:00+00,2015-02-01 17:32:00+00,483,4893,40.732233,345,W 13 St & 6 Ave,40.736494,-73.997044,16219,Subscriber,1961.0,1
4,NaN,W 41 St & 8 Ave,NaN,6156,-73.990026,240,2015-02-01 21:36:00+00,2015-02-01 21:40:00+00,477,6090,40.756405,490,8 Ave & W 33 St,40.751551,-73.993934,18266,Customer,NaN,0


## Task 1 –  Familiarize with SQL Clauses

### Task 1.1 
Sort data by start_station_id, tripduration
- Only checking trips with duration <= 3 hours 

In [6]:
query = '''SELECT * 
FROM fb55.citibike 
WHERE tripduration <= (3*60*60) --3 hours*60mins*60secs
ORDER BY start_station_id asc, tripduration desc;'''

table = get_data(query)

table.head(5)

,the_geom,start_station_name,the_geom_webmercator,field_1,start_station_longitude,tripduration,starttime,stoptime,start_station_id,cartodb_id,start_station_latitude,end_station_id,end_station_name,end_station_latitude,end_station_longitude,bikeid,usertype,birth_year,gender
0,NaN,W 52 St & 11 Ave,NaN,9115,-73.993929,2099,2015-02-03 10:28:00+00,2015-02-03 11:02:00+00,72,9054,40.767272,328,Watts St & Greenwich St,40.724055,-74.009660,18653,Subscriber,1966.0,2
1,NaN,W 52 St & 11 Ave,NaN,2051,-73.993929,1944,2015-02-01 12:39:00+00,2015-02-01 13:12:00+00,72,1983,40.767272,328,Watts St & Greenwich St,40.724055,-74.009660,17849,Customer,NaN,0
2,NaN,W 52 St & 11 Ave,NaN,40355,-73.993929,1914,2015-02-07 09:49:00+00,2015-02-07 10:21:00+00,72,40341,40.767272,328,Watts St & Greenwich St,40.724055,-74.009660,21520,Subscriber,1966.0,2
3,NaN,W 52 St & 11 Ave,NaN,12996,-73.993929,1801,2015-02-04 06:32:00+00,2015-02-04 07:02:00+00,72,12938,40.767272,328,Watts St & Greenwich St,40.724055,-74.009660,15161,Subscriber,1959.0,2
4,NaN,W 52 St & 11 Ave,NaN,11359,-73.993929,1678,2015-02-03 18:21:00+00,2015-02-03 18:49:00+00,72,11298,40.767272,79,Franklin St & W Broadway,40.719116,-74.006667,21500,Subscriber,1964.0,1


### Task 1.2 
Only show the top/last 10 records (aka head and tail in SQL) 

In [7]:
### Top 10 Records
query = '''SELECT *
FROM fb55.citibike 
ORDER BY cartodb_id asc
LIMIT 10;'''

table = get_data(query)

table

,the_geom,start_station_name,the_geom_webmercator,field_1,start_station_longitude,tripduration,starttime,stoptime,start_station_id,cartodb_id,start_station_latitude,end_station_id,end_station_name,end_station_latitude,end_station_longitude,bikeid,usertype,birth_year,gender
0,NaN,8 Ave & W 31 St,NaN,70,-73.994811,801,2015-02-01 00:00:00+00,2015-02-01 00:14:00+00,521,1,40.750450,423,W 54 St & 9 Ave,40.765849,-73.986905,17131,Subscriber,1978,2
1,NaN,E 17 St & Broadway,NaN,71,-73.990093,379,2015-02-01 00:00:00+00,2015-02-01 00:07:00+00,497,2,40.737050,504,1 Ave & E 15 St,40.732219,-73.981656,21289,Subscriber,1993,1
2,NaN,Grand Army Plaza & Central Park S,NaN,72,-73.973715,2474,2015-02-01 00:01:00+00,2015-02-01 00:42:00+00,281,3,40.764397,127,Barrow St & Hudson St,40.731724,-74.006744,18903,Subscriber,1969,2
3,NaN,6 Ave & Broome St,NaN,73,-74.004704,818,2015-02-01 00:01:00+00,2015-02-01 00:15:00+00,2004,4,40.724399,505,6 Ave & W 33 St,40.749013,-73.988484,21044,Subscriber,1985,2
4,NaN,Lawrence St & Willoughby St,NaN,74,-73.986317,544,2015-02-01 00:01:00+00,2015-02-01 00:10:00+00,323,5,40.692362,83,Atlantic Ave & Fort Greene Pl,40.683826,-73.976323,19868,Subscriber,1957,1
5,NaN,Willoughby Ave & Walworth St,NaN,75,-73.953820,717,2015-02-01 00:02:00+00,2015-02-01 00:14:00+00,373,6,40.693317,2002,Wythe Ave & Metropolitan Ave,40.716887,-73.963198,15854,Subscriber,1979,1
6,NaN,W 56 St & 6 Ave,NaN,76,-73.977225,1306,2015-02-01 00:04:00+00,2015-02-01 00:26:00+00,352,7,40.763406,504,1 Ave & E 15 St,40.732219,-73.981656,15173,Subscriber,1983,1
7,NaN,E 4 St & 2 Ave,NaN,77,-73.989780,913,2015-02-01 00:04:00+00,2015-02-01 00:19:00+00,439,8,40.726281,116,W 17 St & 8 Ave,40.741776,-74.001497,17862,Subscriber,1955,1
8,NaN,Washington Pl & Broadway,NaN,78,-73.994046,759,2015-02-01 00:04:00+00,2015-02-01 00:17:00+00,335,9,40.729039,2012,E 27 St & 1 Ave,40.739445,-73.976806,21183,Subscriber,1985,2
9,NaN,Greenwich Ave & 8 Ave,NaN,79,-74.002638,585,2015-02-01 00:05:00+00,2015-02-01 00:15:00+00,284,10,40.739017,444,Broadway & W 24 St,40.742354,-73.989151,14843,Subscriber,1982,1


In [8]:
### Last 10 Records
query = '''SELECT *
FROM fb55.citibike 
ORDER BY cartodb_id desc
LIMIT 10;'''

table = get_data(query)

table

,the_geom,start_station_name,the_geom_webmercator,field_1,start_station_longitude,tripduration,starttime,stoptime,start_station_id,cartodb_id,start_station_latitude,end_station_id,end_station_name,end_station_latitude,end_station_longitude,bikeid,usertype,birth_year,gender
0,NaN,E 4 St & 2 Ave,NaN,46199,-73.989780,917,2015-02-07 23:59:00+00,2015-02-08 00:15:00+00,439,46200,40.726281,417,Barclay St & Church St,40.712912,-74.010202,20998,Subscriber,1965,2
1,NaN,Carmine St & 6 Ave,NaN,46198,-74.002150,548,2015-02-07 23:58:00+00,2015-02-08 00:08:00+00,368,46199,40.730386,334,W 20 St & 7 Ave,40.742388,-73.997262,19540,Subscriber,1983,2
2,NaN,5 Ave & E 29 St,NaN,46197,-73.986831,392,2015-02-07 23:57:00+00,2015-02-08 00:03:00+00,474,46198,40.745168,325,E 19 St & 3 Ave,40.736245,-73.984738,15545,Subscriber,1986,1
3,NaN,5 Ave & E 29 St,NaN,46196,-73.986831,428,2015-02-07 23:57:00+00,2015-02-08 00:04:00+00,474,46197,40.745168,325,E 19 St & 3 Ave,40.736245,-73.984738,16395,Subscriber,1986,2
4,NaN,W 20 St & 8 Ave,NaN,46195,-74.000040,689,2015-02-07 23:57:00+00,2015-02-08 00:08:00+00,470,46196,40.743453,325,E 19 St & 3 Ave,40.736245,-73.984738,15585,Subscriber,1953,1
5,NaN,1 Ave & E 44 St,NaN,46194,-73.969053,1422,2015-02-07 23:57:00+00,2015-02-08 00:20:00+00,455,46195,40.750020,265,Stanton St & Chrystie St,40.722293,-73.991475,20184,Subscriber,1960,2
6,NaN,E 20 St & 2 Ave,NaN,46193,-73.982050,993,2015-02-07 23:56:00+00,2015-02-08 00:13:00+00,461,46194,40.735877,295,Pike St & E Broadway,40.714067,-73.992939,16722,Subscriber,1974,1
7,NaN,Warren St & Church St,NaN,46192,-74.009106,1165,2015-02-07 23:56:00+00,2015-02-08 00:15:00+00,152,46193,40.714740,325,E 19 St & 3 Ave,40.736245,-73.984738,16978,Subscriber,1959,1
8,NaN,W 47 St & 10 Ave,NaN,46191,-73.993012,1248,2015-02-07 23:56:00+00,2015-02-08 00:17:00+00,495,46192,40.762699,432,E 7 St & Avenue A,40.726218,-73.983799,16300,Subscriber,1984,1
9,NaN,W 17 St & 8 Ave,NaN,46190,-74.001497,306,2015-02-07 23:55:00+00,2015-02-08 00:01:00+00,116,46191,40.741776,494,W 26 St & 8 Ave,40.747348,-73.997236,14736,Subscriber,1983,2


### Task 1.3
List all unique start_station_id values

In [9]:
query = '''SELECT distinct(start_station_id)
FROM fb55.citibike 
Where start_station_id is not null;'''

table = get_data(query)

table

,start_station_id,Unnamed: 1
0,120,NaN
1,285,NaN
2,251,NaN
3,195,NaN
4,453,NaN
5,144,NaN
6,266,NaN
7,263,NaN
8,216,NaN
9,328,NaN


### Task 1.4
Aggregation functions:
- Count the number of trips (aka wc -l in SQL)
- Find the average/min/max trip duration

In [10]:
query = '''SELECT count(cartodb_id) as NUM_trips, min(tripduration) as MIN_tripduration, max(tripduration) as MAX_tripduration, avg(tripduration)as AVG_tripduration
FROM fb55.citibike ;'''

table = get_data(query)

table

,num_trips,min_tripduration,max_tripduration,avg_tripduration
0,46200,60,43016,675.865823


## Task 2 – Working with Date/Time

### Task 2.1 
1) Selecting trips started on Feb-02-2015 only 


In [11]:
query = '''SELECT cartodb_id, starttime
FROM fb55.citibike
WHERE CAST(starttime AS Date) = '2015-02-02';'''

table = get_data(query)

table

,cartodb_id,starttime
0,7367,2015-02-02 17:44:00+00
1,7712,2015-02-02 20:12:00+00
2,6920,2015-02-02 11:07:00+00
3,6477,2015-02-02 00:40:00+00
4,6478,2015-02-02 00:40:00+00
5,6526,2015-02-02 06:20:00+00
6,6442,2015-02-02 00:02:00+00
7,6443,2015-02-02 00:03:00+00
8,6444,2015-02-02 00:04:00+00
9,6445,2015-02-02 00:06:00+00


### Task 2.2.1
Selecting trips started on the weekends

In [12]:
## find the trips that were started during the weekends
query = '''SELECT cartodb_id, starttime, case when to_char(starttime,'DY') in ('SAT','SUN') then 'Weekend' end tag, tripduration 
FROM fb55.citibike
WHERE to_char(starttime,'DY') in ('SAT','SUN');'''

table = get_data(query)

table

,cartodb_id,starttime,tag,tripduration
0,107,2015-02-01 01:23:00+00,Weekend,1090
1,1088,2015-02-01 10:55:00+00,Weekend,682
2,2759,2015-02-01 13:59:00+00,Weekend,751
3,4893,2015-02-01 17:28:00+00,Weekend,272
4,6090,2015-02-01 21:36:00+00,Weekend,240
5,39876,2015-02-07 08:13:00+00,Weekend,296
6,41501,2015-02-07 12:35:00+00,Weekend,600
7,41585,2015-02-07 12:47:00+00,Weekend,1203
8,41796,2015-02-07 13:16:00+00,Weekend,520
9,42307,2015-02-07 14:18:00+00,Weekend,487


### Task 2.2.2
- What are average trip duration during weekends? 

In [13]:
## find the average tripdurationthat were started during the weekends
query = '''SELECT count(cartodb_id) as num_Trips,avg(tripduration) as AVG_weekendTrips 
FROM fb55.citibike
WHERE to_char(starttime,'DY') in ('SAT','SUN');'''

table = get_data(query)

table

,num_trips,avg_weekendtrips
0,13231,662.942181


### Task 2.3.1
3) Can we do the same for weekday?

In [14]:
## find the trips that were started during the weekdays
query = '''SELECT cartodb_id, starttime, case when to_char(starttime,'DY') NOT in ('SAT','SUN') then 'Weekday' end tag, tripduration 
FROM fb55.citibike
WHERE to_char(starttime,'DY') not in ('SAT','SUN');'''

table = get_data(query)

table

,cartodb_id,starttime,tag,tripduration
0,7367,2015-02-02 17:44:00+00,Weekday,376
1,7712,2015-02-02 20:12:00+00,Weekday,1217
2,8263,2015-02-03 08:02:00+00,Weekday,441
3,9375,2015-02-03 12:31:00+00,Weekday,1475
4,10411,2015-02-03 16:45:00+00,Weekday,420
5,10867,2015-02-03 17:37:00+00,Weekday,876
6,11300,2015-02-03 18:21:00+00,Weekday,640
7,12114,2015-02-03 20:26:00+00,Weekday,571
8,12152,2015-02-03 20:37:00+00,Weekday,821
9,12500,2015-02-03 22:20:00+00,Weekday,376


### Task 2.3.2
find the average tripduration for weekdays

In [15]:
## find the average trip duration on weekdays
query = '''SELECT count(cartodb_id) as num_trips, avg(tripduration) as AVG_tripduration 
FROM fb55.citibike
WHERE to_char(starttime,'DY') NOT in ('SAT','SUN');'''

table = get_data(query)

table

,num_trips,avg_tripduration
0,32969,681.052292


## Task 3 – Working with Space

### Task 3.1
1) Showing the list of start station locations
- Using GROUP BY 

In [18]:
query = '''SELECT start_station_id, start_station_name
FROM fb55.citibike
GROUP BY start_station_id, start_station_name;'''

table = get_data(query)

table

,start_station_id,start_station_name
0,257,Lispenard St & Broadway
1,453,W 22 St & 8 Ave
2,291,Madison St & Montgomery St
3,387,Centre St & Chambers St
4,364,Lafayette Ave & Classon Ave
5,354,Emerson Pl & Myrtle Ave
6,491,E 24 St & Park Ave S
7,355,Bayard St & Baxter St
8,146,Hudson St & Reade St
9,279,Peck Slip & Front St


### Task 3.2
2) Showing the number of trips started per station;


In [19]:
query = '''SELECT start_station_id, start_station_name, count(cartodb_id) as num_Trips
FROM fb55.citibike
GROUP BY start_station_id, start_station_name;'''

table = get_data(query)

table

,start_station_id,start_station_name,num_trips
0,257,Lispenard St & Broadway,123
1,453,W 22 St & 8 Ave,218
2,291,Madison St & Montgomery St,108
3,387,Centre St & Chambers St,159
4,364,Lafayette Ave & Classon Ave,13
5,354,Emerson Pl & Myrtle Ave,30
6,491,E 24 St & Park Ave S,288
7,355,Bayard St & Baxter St,136
8,146,Hudson St & Reade St,101
9,279,Peck Slip & Front St,89


### Task 3.3
3) … but only for stations within 500m of Time Square!
- The coordinates of Time Square is (40.7577,-73.9857)

#### from GIS stackexchange
https://gis.stackexchange.com/questions/2951/algorithm-for-offsetting-a-latitude-longitude-by-some-amount-of-meters
If your displacements aren't too great (less than a few kilometers) and you're not right at the poles, use the quick and dirty estimate that 111,111 meters (111.111 km) in the y direction is 1 degree (of latitude) and 111,111 * cos(latitude) meters in the x direction is 1 degree (of longitude).

In [21]:
## find 500 m / 111,111 m for the fraction of 1 degree

d = 500/111111
d

0.0045000045000045

In [22]:
query = '''SELECT start_station_id, start_station_name, count(cartodb_id)
FROM fb55.citibike
WHERE (start_station_longitude <= (-73.9857 + 0.0045000045000045) AND start_station_longitude >= (-73.9857 - 0.0045000045000045))
AND (start_station_latitude <= (40.7577 + 0.0045000045000045) AND start_station_latitude >= (40.7577 - 0.0045000045000045))
GROUP BY start_station_id, start_station_name;'''

table = get_data(query)

table

,start_station_id,start_station_name,count
0,465,Broadway & W 41 St,251
1,173,Broadway & W 49 St,213
2,477,W 41 St & 8 Ave,507
3,493,W 45 St & 6 Ave,141
4,2021,W 45 St & 8 Ave,141
5,524,W 43 St & 6 Ave,112


## Task 4 — Putting it all together

### Task 4.1
Find the station that had the longest average trip duration during
weekends and within 500m of TimeSquare! 

In [23]:
query = '''SELECT start_station_id, start_station_name, avg(tripduration) as AVG_tripduration
FROM fb55.citibike
WHERE (start_station_longitude <= (-73.9857 + 0.0045000045000045) AND start_station_longitude >= (-73.9857 - 0.0045000045000045))
AND (start_station_latitude <= (40.7577 + 0.0045000045000045) AND start_station_latitude >= (40.7577 - 0.0045000045000045))
AND to_char(starttime,'DY') in ('SAT','SUN')
GROUP BY start_station_id , start_station_name
ORDER BY AVG_tripduration desc
LIMIT 1;'''

table = get_data(query)

table

,start_station_id,start_station_name,avg_tripduration
0,173,Broadway & W 49 St,1010.104167


# extra credit:
create lines for trips started from stations within 500m of Times
Squares and lasted less than 2 hours. The number of trips per each
pair of stations are output as attributes of these lines.

In [29]:
query = '''SELECT start_station_id, start_station_name, end_station_id, end_station_name, count(cartodb_id) as num_trips
FROM fb55.citibike
WHERE (start_station_longitude <= (-73.9857 + 0.0045000045000045) AND start_station_longitude >= (-73.9857 - 0.0045000045000045))
AND (start_station_latitude <= (40.7577 + 0.0045000045000045) AND start_station_latitude >= (40.7577 - 0.0045000045000045))
AND tripduration < 2*60*60
GROUP BY start_station_id, start_station_name, end_station_id, end_station_name
ORDER BY num_trips desc;'''

table = get_data(query)

table

,start_station_id,start_station_name,end_station_id,end_station_name,num_trips
0,477,W 41 St & 8 Ave,318,E 43 St & Vanderbilt Ave,31
1,477,W 41 St & 8 Ave,359,E 47 St & Park Ave,21
2,477,W 41 St & 8 Ave,517,Pershing Square South,17
3,477,W 41 St & 8 Ave,519,Pershing Square North,15
4,173,Broadway & W 49 St,318,E 43 St & Vanderbilt Ave,15
5,524,W 43 St & 6 Ave,318,E 43 St & Vanderbilt Ave,14
6,465,Broadway & W 41 St,492,W 33 St & 7 Ave,14
7,477,W 41 St & 8 Ave,153,E 40 St & 5 Ave,14
8,493,W 45 St & 6 Ave,492,W 33 St & 7 Ave,14
9,173,Broadway & W 49 St,538,E 48 St & Madison Ave,13
